# В этом файле считаем косинусное расстояние между клиентами

Взяли клиентов, у которых больше 6-ти прочитанных книг. Посчитали и записали в файл `cosine_dim.txt`

In [1]:
import pandas as pd
import math
import numpy as np
import scipy.spatial.distance as ds
from scipy.sparse import csr_matrix, save_npz, load_npz
from tqdm import tqdm
import json
import pickle
import time

In [2]:
book_id = pd.read_csv('datasets/books/book_id_map.csv')
book_title = pd.read_csv('datasets/books/book_titles.csv')
call_meta = pd.read_csv('datasets/books/collaborative_book_metadata.csv')
call_book = pd.read_csv('datasets/books/collaborative_books_df.csv')
user_id = pd.read_csv('datasets/books/user_id_map.csv')

In [2]:
call_book = call_book.sort_values('user_id_mapping')

NameError: name 'call_book' is not defined

In [26]:
((pd.read_csv('datasets/books/collaborative_books_df.csv').sort_values('user_id_mapping').user_id_mapping.unique))

<bound method Series.unique of 18356         0
178192        0
59380         0
61307         0
192251        0
          ...  
153944    84560
156450    84561
1816      84562
73990     84563
116126    84564
Name: user_id_mapping, Length: 196296, dtype: int64>

In [3]:
max_user_id = 84564

In [4]:
call_book_short = pd.read_excel('collaborative_books_short.xlsx')

In [5]:
call_book = call_book_short

In [12]:
ds.cosine([1, 0], [0, 1])

1.0

https://habr.com/ru/articles/150399/ - коллаборативная фильтрация

In [6]:
# Получаем словарь, где по id юзера указаны id прочитанных им книг
read_by_user = {}
user_id_mapping = set(call_book['user_id_mapping'])
for i in user_id_mapping:
    temp_df = call_book.loc[call_book['user_id_mapping'] == i]
    read_by_user.update({i: sorted(list(temp_df['book_id']))})


In [6]:
list_keys = []
for key in read_by_user.keys():
    if len(read_by_user[key]) > 6:
        list_keys.append(key)
len(list_keys)

3897

## запись в файл, если надо

In [33]:
# call_book_short = call_book[call_book['user_id_mapping'] == 0]
temp_df = []
for i in list_keys:
    temp_df.append(call_book[call_book['user_id_mapping'] == i])
    # call_book_short = call_book_short.merge(temp, how='inner')

In [34]:
call_book_short = pd.concat(temp_df)

In [36]:
call_book_short.to_excel('collaborative_books_short.xlsx')

In [38]:
with open("read_by_user.pickle", "wb") as file:
    pickle.dump(read_by_user, file)

In [29]:
with open("read_by_user.pickle", "rb") as file:
    read_by_user = pickle.load(file)

Запись в файлы для передачи в плюсы

In [21]:
with open("read_by_user.txt", 'w') as file:
    for key in read_by_user.keys():
        file.write(' '.join(list(map(str, read_by_user[key]))) + '\n')

In [22]:
with open("read_by_user_keys.txt", 'w') as file:
    for key in read_by_user.keys():
        file.write(str(key) + '\n')

Создаем разреженную матрицу, где все значения идут по порядку. Отсортировано по id_user_mapping, затем по book_id. 
Значениями матрицы являются оценки пользователя

In [7]:
row_indices = list()
column_indices = list()
for key in read_by_user.keys():
    row_indices.extend([key]*len(read_by_user[key]))
    column_indices.extend(read_by_user[key])

grade = np.array(call_book.sort_values(['user_id_mapping', 'book_id'])['Predicted Rating'])
row_indices = np.array(row_indices)
column_indices = np.array(column_indices)

Строки - user_id_mapping, столбцы - book_id. Значение - Predicted Rating

In [8]:
grade_matrix = csr_matrix((grade, (row_indices, column_indices)))

In [8]:
len(grade_matrix.nonzero()[0])

31700

In [17]:
# len(grade_matrix.nonzero()[0])
# собираемся записать в датафрейм только ненулевые столбцы
# grade_df = pd.DataFrame()
grade_list = []
# берем все строки, но удааляем полностью нулевые стоблцы
for i in tqdm(range(max(grade_matrix.nonzero()[0]) + 1)):
    grade_list.append(grade_matrix.getrow(i).toarray()[0][grade_matrix.nonzero()[1]])

100%|██████████| 77764/77764 [06:11<00:00, 209.15it/s]


In [19]:
with open('grade_list.txt', 'w') as file:
    for grade in grade_list:
        file.write(str(grade) + '\n')

In [ ]:
with open('grade_list.txt') as file:
    for line in file:
        grade_list.append(eval(line))

In [20]:
len(grade_list)

77764

Пробуем создать генератор для уменьшения затрат памяти

In [9]:
def gennerator_grade_list():
    for i in tqdm(range(max_user_id)):
        yield grade_matrix.getrow(i).toarray()[0][grade_matrix.nonzero()[1]]

In [10]:
len(grade_list[0])

31700

In [23]:
save_npz('grade_matrix_short.npz', grade_matrix)

In [20]:
grade_matrix = load_npz('grade_matrix.npz')

In [8]:
indexes = []
with open("output_key_index.txt") as read_file:
    for line in tqdm(read_file):
        # temp = tuple(map(int, line[1:-2].split(', ')))
        temp = line[:-1].replace(',', '').replace('(', '').replace(')', '')
        indexes.append(temp)

0it [00:00, ?it/s]

2008497it [00:01, 1394612.42it/s]


Создаем генератор для индексов

In [10]:
def get_index():
    with open("output_key_index.txt") as read_file:
        for line in tqdm(read_file):
            # temp = tuple(map(int, line[1:-2].split(', ')))
            inx = line[:-1].replace(',', '').replace('(', '').replace(')', '')
            yield inx

In [14]:
with open("key_index.txt", 'w') as file:
    for line in tqdm(indexes):
        file.write(str(line) + '\n')

100%|██████████| 2008497/2008497 [00:01<00:00, 1702021.78it/s]


In [20]:
len(grade_matrix.getrow(i).toarray()[0])

30555489

In [36]:
grade_matrix.nonzero()[1].shape

(31700,)

Рачет косинусной шляпы

In [24]:
# cosine_grade = []
with open('output_key_index_3000.txt') as file:
    with open('cosine_dim.txt', 'w') as write_file:
        for line in tqdm(file):
            i, j = list(map(int, line.split()))
            cosine = (ds.cosine(grade_list[i], grade_list[j]))
            write_file.write(str(i) + ' ' + str(j) + ' ' + str(cosine) + '\n')

796274it [06:12, 2138.91it/s]


In [35]:
cosine_list = []
with open('cosine_dim.txt') as file:
    for line in tqdm(file):
        cosine_list.append(line.split())

796274it [00:01, 622090.12it/s]


In [46]:
cosine_dim = pd.DataFrame(columns=['x', 'y', 'cosine'], data=(cosine_list))

In [54]:
cosine_dim.sort_values('cosine', ascending=False).head(15)

,x,y,cosine
785189,15426,16245,0.9957953740295578
193731,25581,19022,0.9953850082498844
388366,26769,52630,0.9950873408474369
388559,26769,15619,0.9949771370680286
315348,9887,19239,0.9949444673104856
1574,16393,25152,0.9947803337081287
112902,606,30329,0.9945473897420658
732719,30329,23517,0.9945059991361832
323134,26314,26769,0.9944611466610263
34619,16547,2775,0.9944609955589867


In [15]:
print(i, j)

8225 77763


In [16]:
len(grade_list)

77763